In [1]:
import sys
sys.path.append('../')

In [3]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, MetaData, Table, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

In [4]:
USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

PATH_OUTPUT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\ESTIMATIVAS\V06'

metadata = MetaData()

In [5]:
# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_estimativa():
    try:
        query = "select unidad_01, unidad_03, unidad_05, sum(area) as area from catastro_iag.estimativa group by unidad_01, unidad_03, unidad_05"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

# obtiene un resumen de la capa de estimativa (idd - area)
def get_resumen_cosecha():
    try:
        query = "select unidad_01, unidad_03, unidad_05, sum(area) as area from catastro_iag.cosecha_2025 group by unidad_01, unidad_03, unidad_05"
        df = pd.read_sql(query, ENGINE)
        return df
    except Exception as e:
        print("❌ Error al leer datos de COSECHA 2025:", e)
        return pd.DataFrame()
    return None

def eliminar_propiedad_estimativa_db(cod_prop):
    """
    Elimina registros con el cod_prop enviado de la tabla estimativa
    """

    sql_query = text(f"""
        DELETE FROM catastro_iag.estimativa
        WHERE unidad_01 = :cod_prop
    """)

    try:
        with ENGINE.connect() as conn:
            result = conn.execute(sql_query, {"cod_prop": cod_prop})
            conn.commit()  # Asegura que se guarden los cambios
        print(f"Se eliminaron {result.rowcount} registros ESTIMATIVAS de la propiedad: {cod_prop}")
        return result.rowcount
    except Exception as e:
        print(f"Error al ejecutar la eliminación propiedad de ESTIMATIVAS: {e}")
        return None
    
def eliminar_propiedad_SHP_COSECHA_db(cod_prop):
    """
    Elimina registros con el cod_prop enviado de la tabla shp_cosecha
    """

    sql_query = text(f"""
        DELETE FROM catastro_iag.cosecha_2025 
        WHERE unidad_01 = :cod_prop
    """)

    try:
        with ENGINE.connect() as conn:
            result = conn.execute(sql_query, {"cod_prop": cod_prop})
            conn.commit()  # Asegura que se guarden los cambios
        print(f"Se eliminaron {result.rowcount} registros de SHP_COSECHA de la propiedad: {cod_prop}")
        return result.rowcount
    except Exception as e:
        print(f"Error al ejecutar la eliminación de SHP COSECHA: {e}")
        return None

In [6]:
df_estimativa = get_resumen_estimativa()
df_estimativa['idd'] = df_estimativa['unidad_01'].astype(str) + '|' + df_estimativa['unidad_03'].astype(str) + '|' + df_estimativa['unidad_05']
df_estimativa = df_estimativa[['idd', 'area']]
output_path = "filas_estimativas.xlsx"
df_estimativa.to_excel(output_path, index=False)
df_estimativa

,idd,area
0,1|388|L1,3.760333
1,1|388|L10.2,7.831933
2,1|388|L11,8.721908
3,1|388|L13.2,7.381695
4,1|388|L13.3,1.618728
...,...,...
10605,2302|1447|L5,16.220018
10606,2302|1447|L6,11.651393
10607,2302|1447|L7,16.645492
10608,2304|4915|L1,6.824405


In [7]:
df_cosecha = get_resumen_cosecha()
df_cosecha['idd'] = df_cosecha['unidad_01'].astype(str) + '|' + df_cosecha['unidad_03'].astype(str) + '|' + df_cosecha['unidad_05']
df_cosecha = df_cosecha[['idd', 'area']]
output_path = "filas_cosecha.xlsx"
df_cosecha.to_excel(output_path, index=False)
df_cosecha

,idd,area
0,830|1686|L1,2.356509
1,1695|40251|L1,19.618234
2,632|6578|L1,6.776382
3,2006|12314|L1,3.023093
4,2166|40185|L2,2.358242
...,...,...
10605,320|13123|L12.2,0.571402
10606,326|40141|L7.1.2,1.579672
10607,1854|13658|A3,2.531835
10608,855|12118|L9,0.865902


In [8]:
df_merged = pd.merge(
    df_estimativa,
    df_cosecha,
    on="idd",
    how="outer",
    suffixes=("_estimativa", "_cosecha")
)

In [9]:
df_merged

,idd,area_estimativa,area_cosecha
0,1000|16213|L1.1,2.551941,2.484536
1,1000|16213|L1.3,5.839942,5.763180
2,1000|16213|L2.1,2.950992,3.015121
3,1000|16213|L2.2,3.313293,3.241613
4,1000|16213|L3,4.139541,4.265272
...,...,...,...
10605,9|18115|L2,1.641776,1.714524
10606,9|18115|L3.1,1.760447,1.639247
10607,9|18115|L3.2,1.987573,1.772982
10608,9|18115|L4,4.102529,4.286313


In [10]:
df_merged['dif'] = abs(df_merged['area_estimativa'] - df_merged['area_cosecha'])

In [11]:
df_merged

,idd,area_estimativa,area_cosecha,dif
0,1000|16213|L1.1,2.551941,2.484536,0.067405
1,1000|16213|L1.3,5.839942,5.763180,0.076762
2,1000|16213|L2.1,2.950992,3.015121,0.064128
3,1000|16213|L2.2,3.313293,3.241613,0.071680
4,1000|16213|L3,4.139541,4.265272,0.125731
...,...,...,...,...
10605,9|18115|L2,1.641776,1.714524,0.072748
10606,9|18115|L3.1,1.760447,1.639247,0.121201
10607,9|18115|L3.2,1.987573,1.772982,0.214591
10608,9|18115|L4,4.102529,4.286313,0.183784


In [12]:
output_path = "filas_all.xlsx"
df_merged.to_excel(output_path, index=False)

In [13]:
df_nan_mas = df_merged[
    (df_merged["area_estimativa"].isna()) |
    (df_merged["area_cosecha"].isna()) |
    (df_merged["dif"] > 2)
]
df_nan_mas

,idd,area_estimativa,area_cosecha,dif
236,1063|16111|L1,43.766775,41.124054,2.642721
715,114|41970|L2.2,43.498288,40.951706,2.546582
1115,122|15501|N5,44.415798,42.108779,2.307020
4901,217|794|FLL10,21.661807,19.361666,2.300141
5364,239|9677|L2,23.546671,21.288255,2.258416
5813,275|18356|L25,24.078562,21.821622,2.256940
6203,300|2899|L52.2,14.483370,11.655944,2.827426
6337,30|41594|ER-L23,65.281390,59.020237,6.261153
6598,326|40141|L3.1,26.212695,23.737379,2.475315
7483,480|41594|Ca-2,59.960973,57.214863,2.746110


In [14]:
output_path = "filas_nan.xlsx"
df_nan_mas.to_excel(output_path, index=False)

In [15]:
df_nan_mas

,idd,area_estimativa,area_cosecha,dif
236,1063|16111|L1,43.766775,41.124054,2.642721
715,114|41970|L2.2,43.498288,40.951706,2.546582
1115,122|15501|N5,44.415798,42.108779,2.307020
4901,217|794|FLL10,21.661807,19.361666,2.300141
5364,239|9677|L2,23.546671,21.288255,2.258416
5813,275|18356|L25,24.078562,21.821622,2.256940
6203,300|2899|L52.2,14.483370,11.655944,2.827426
6337,30|41594|ER-L23,65.281390,59.020237,6.261153
6598,326|40141|L3.1,26.212695,23.737379,2.475315
7483,480|41594|Ca-2,59.960973,57.214863,2.746110


In [16]:
df_split = df_nan_mas.copy()
df_split[['unidad_01', 'unidad_03', 'unidad_05']] = df_split['idd'].str.split('|', expand=True)

In [17]:
df_split

,idd,area_estimativa,area_cosecha,dif,unidad_01,unidad_03,unidad_05
236,1063|16111|L1,43.766775,41.124054,2.642721,1063,16111,L1
715,114|41970|L2.2,43.498288,40.951706,2.546582,114,41970,L2.2
1115,122|15501|N5,44.415798,42.108779,2.307020,122,15501,N5
4901,217|794|FLL10,21.661807,19.361666,2.300141,217,794,FLL10
5364,239|9677|L2,23.546671,21.288255,2.258416,239,9677,L2
5813,275|18356|L25,24.078562,21.821622,2.256940,275,18356,L25
6203,300|2899|L52.2,14.483370,11.655944,2.827426,300,2899,L52.2
6337,30|41594|ER-L23,65.281390,59.020237,6.261153,30,41594,ER-L23
6598,326|40141|L3.1,26.212695,23.737379,2.475315,326,40141,L3.1
7483,480|41594|Ca-2,59.960973,57.214863,2.746110,480,41594,Ca-2


In [18]:
lista_cods = list(set(df_split['unidad_01']))
lista_cods = [int(i) for i in lista_cods]
lista_cods.sort()
print('Cantidad:', len(lista_cods))
print(lista_cods)

Cantidad: 13
[30, 114, 122, 217, 239, 275, 300, 326, 480, 592, 629, 949, 1063]


In [19]:
len(list(set(df_split['unidad_01'])))

13

In [20]:
lista_cods = [30, 110, 114, 122, 217, 239, 275, 300, 326, 480, 592, 629, 630, 673, 766, 773, 782, 877, 949, 1052, 1063, 1222, 1314, 1454, 1507, 1567, 1656, 2158, 2189]

In [21]:
for i in lista_cods:
    eliminar_propiedad_estimativa_db(i)

Se eliminaron 8714 registros ESTIMATIVAS de la propiedad: 30
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 110
Se eliminaron 7249 registros ESTIMATIVAS de la propiedad: 114
Se eliminaron 6030 registros ESTIMATIVAS de la propiedad: 122
Se eliminaron 3130 registros ESTIMATIVAS de la propiedad: 217
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 239
Se eliminaron 4677 registros ESTIMATIVAS de la propiedad: 275
Se eliminaron 12840 registros ESTIMATIVAS de la propiedad: 300
Se eliminaron 2269 registros ESTIMATIVAS de la propiedad: 326
Se eliminaron 5665 registros ESTIMATIVAS de la propiedad: 480
Se eliminaron 3370 registros ESTIMATIVAS de la propiedad: 592
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 629
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 630
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 673
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 766
Se eliminaron 0 registros ESTIMATIVAS de la propiedad: 773
Se eliminaron 0 registros EST

In [56]:
resumen_estimativa = get_resumen_estimativa()
resumen_cosecha = get_resumen_cosecha()

print('Total Resumen Estimativa:', resumen_estimativa['area'].sum())
print('Total Resumen SHPCosecha:', resumen_cosecha['area'].sum())

Total Resumen Estimativa: 37476.39200378607
Total Resumen SHPCosecha: 55483.70074953324


In [15]:
# identifiar lotes que estan en ESTIMATIVAS y no en SHP_Cosecha
faltantes = resumen_estimativa[~resumen_estimativa['idd'].isin(resumen_cosecha['idd'])].copy()
if len(faltantes) > 0:
    # Dividir la columna idd en 3 columnas nuevas ('unidad_01', 'unidad_03', 'unidad_05')
    faltantes[['unidad_01', 'unidad_03', 'unidad_05']] = faltantes['idd'].str.split('|', expand=True)
    # extraer lista de unicos de codigos de propiedad
    codigos_prop = list(set(faltantes['unidad_01']))
    # convertir lista de str a int
    codigos_prop = [int(i) for i in codigos_prop]
    codigos_prop.sort()
    codigos_prop
else:
    codigos_prop = []
    print('No existen lotes que esten en ESTIMATIVAS y no en SHP COSECHA')

No existen lotes que esten en ESTIMATIVAS y no en SHP COSECHA


In [16]:
codigos_prop

[]

In [17]:
for i in codigos_prop:
    eliminar_propiedad_db(i)

In [11]:
resumen_estimativa = get_resumen_estimativa()
resumen_cosecha = get_resumen_cosecha()

print('Total Resumen Estimativa:', resumen_estimativa['area'].sum())
print('Total Resumen SHPCosecha:', resumen_cosecha['area'].sum())

Total Resumen Estimativa: 54030.595134443785
Total Resumen SHPCosecha: 55483.70074953324


In [12]:
# identifiar lotes que estan en ESTIMATIVAS y no en SHP_Cosecha
faltantes = resumen_cosecha[~resumen_cosecha['idd'].isin(resumen_estimativa['idd'])].copy()
if len(faltantes) > 0:
    # Dividir la columna idd en 3 columnas nuevas ('unidad_01', 'unidad_03', 'unidad_05')
    faltantes[['unidad_01', 'unidad_03', 'unidad_05']] = faltantes['idd'].str.split('|', expand=True)
    # extraer lista de unicos de codigos de propiedad
    codigos_prop = list(set(faltantes['unidad_01']))
    # convertir lista de str a int
    codigos_prop = [int(i) for i in codigos_prop]
    codigos_prop.sort()
    codigos_prop
else:
    codigos_prop = []
    print('No existen lotes que esten en SHP COSEHCA y no en ESTIMATIVA')

In [13]:
codigos_prop

[62,
 123,
 141,
 205,
 408,
 504,
 560,
 592,
 630,
 673,
 773,
 782,
 877,
 1222,
 1507,
 1567,
 1656,
 1801,
 2189]

In [129]:
for i in codigos_prop:
    eliminar_propiedad_db(i)